In [19]:
import praw
import pandas as pd
import datetime as dt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/akshitaggarwal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Setting up Reddit AuthO

In [11]:
reddit = praw.Reddit(client_id='WK7Bb4kSnRez8g',
                     client_secret='NYWBOS04ouTSPO0cKhsMLqs-SWQ',
                     user_agent='Reddit_India_Data')
india_subreddit = reddit.subreddit('india')

In [12]:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]
post_dict = []

In [15]:
def readableTime(created):
    return dt.datetime.utcfromtimestamp(created)


#To convert data elements into string objects
def toStr(item):
    return str(item)

_replace = re.compile('[/(){}\|@,;]')
badSymbols = re.compile('[^0-9a-z #+_]')
stopWords = set(stopwords.words('english'))

#To clean text within dataset
def cleanText(string):
    string = BeautifulSoup(string, "lxml").string
    string = string.lower() 
    string = _replace.sub(' ', string)
    string = badSymbols.sub('', string)
    string = ' '.join(word for word in string.split() if word not in stopWords)
    return string

In [14]:
for flair in flairs:
    submissions = india_subreddit.search(f"flair_name:{flair}",limit=100)
    for submission in submissions:
        post_dict.append([submission.title,
                submission.id,
                submission.link_flair_text,
                submission.author,
                submission.url,
                submission.num_comments,
                submission.score,
                submission.created])
post_dict = pd.DataFrame(post_dict,columns=['title', 'id', 'flair', 'author', 'url', 'num_comments', 'score', 'created'])
topics = pd.DataFrame(post_dict)
#Sorting data based in ascending order of flair
topics.sort_values('flair', inplace=True)
#Changing Unix time to UTC time.
topics['timestamp'] = topics['created'].apply(readableTime)
del topics['created']
#Deleting any duplicate rows
topics.drop_duplicates(subset=None, inplace=True)
topics.to_csv('Data.csv', index=False)

# DB Cleaning and Text Pre Processing

In [18]:
df = pd.read_csv('Data.csv')
df.head()
df['title'] =df['title'].apply(toStr)
df['title'] =df['title'].apply(cleanText)
feature_Grp = df['title'] + df['url']
df = df.assign(feature_Grp = feature_Grp)
df.to_csv('Final_formatted_data.csv', index=False)